In [18]:
import pandas as pd
import numpy as np
import pyspark
from pyspark.sql import *
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext # https://spark.apache.org/docs/1.6.1/sql-programming-guide.html
from os.path import join, abspath

# https://stackoverflow.com/questions/21138751/spark-java-lang-outofmemoryerror-java-heap-space/22742982#22742982
# https://luminousmen.com/post/spark-partitions
# https://spark.apache.org/docs/latest/sql-data-sources-hive-tables.html
warehouse_location = abspath('spark-warehouse/test')
# Create the session
conf = (SparkConf()
    .set("spark.ui.port", "4041")
#    .set('spark.executor.memory', '6G')
#    .set('spark.driver.memory', '6G')
#    .set('spark.storage.memoryFraction', '.5')
#    .set('spark.driver.maxResultSize', '2G')
    .set('spark.sql.warehouse.dir', warehouse_location)
#    .set("spark.dynamicAllocation.enabled", "true")   
#    .set("spark.executor.cores", "4")
#    .set("spark.dynamicAllocation.minExecutors","1")
#    .set("spark.dynamicAllocation.maxExecutors","5")
#    .set("spark.sql.legacy.allowCreatingManagedTableUsingNonemptyLocation","true")
       )

# Create the context
sc = pyspark.SparkContext(conf=conf)
spark = (SparkSession.builder
    .appName('Spark Practice')
    .config("hive.metastore.uris", "thrift://localhost:9083")
    .enableHiveSupport()
    .getOrCreate())
sqlContext = SQLContext(sc)

In [17]:
# spark.stop()

In [19]:
spark.catalog.listDatabases()

[Database(name='default', description='default database', locationUri='/home/jovyan/cse451/spark-warehouse/test')]

In [20]:
master = spark.read.parquet("spark-warehouse/test/open990.db/master/")
grants = spark.read.parquet("spark-warehouse/test/open990.db/grants/")
governance = spark.read.parquet("spark-warehouse/test/open990.db/governance/")
foundations = spark.read.parquet("spark-warehouse/test/open990.db/foundations/")
executive_foundations = spark.read.parquet("spark-warehouse/test/open990.db/executive_foundations/")
executive_charity = spark.read.parquet("spark-warehouse/test/open990.db/executive_charity/")
contractor = spark.read.parquet("spark-warehouse/test/open990.db/contractor/")

In [21]:
master.createOrReplaceTempView("master")
grants.createOrReplaceTempView("grant")
governance.createOrReplaceTempView("governance")
foundations.createOrReplaceTempView("foundation")
executive_foundations.createOrReplaceTempView("exec_foundation")
executive_charity.createOrReplaceTempView("exec_charity")
contractor.createOrReplaceTempView("contractor")


In [22]:
spark.sql('SHOW TABLES').show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|        |     contractor|       true|
|        |   exec_charity|       true|
|        |exec_foundation|       true|
|        |     foundation|       true|
|        |     governance|       true|
|        |          grant|       true|
|        |         master|       true|
+--------+---------------+-----------+



In [14]:
spark.sql('''
SELECT *
FROM contractor
LIMIT 5
''').toPandas()

,ein,name_org,tax_date_begin,tax_date_end,tax_yr,doing_business_as,phone,website,address,city,...,state_contractor_5,zip_contractor_5,country_contractor_5,services_contractor_5,amt_paid_contractor_5,contractor_100k_ct,record_id,schema_version,irs_efile_id,random_string
0,43234724,WILD CAREINC,2016-01-01,2016-12-31,2016,None,+1 (508) 240-2255,WILDCARECAPECOD.ORG,10 SMITH LANE,EASTHAM,...,None,None,None,None,NaN,NaN,043234724_201612,2016v3.0,201722999349300547,NDcyMjU3Mzk0MjUwNzU4NTE1MQ
1,46144180,NEW ENGLAND VILLAGE INC,2015-07-01,2016-06-30,2016,None,+1 (781) 293-5461,WWW.NEWENGLANDVILLAGE.ORG,664 SCHOOL STREET,PEMBROKE,...,None,None,None,None,NaN,1.0,046144180_201606,2015v3.0,201710109349300631,NTcyNDAyNDQ3MTQ5MjM5Mjk0Mw
2,201226494,UNIVERSITY OF FLORIDA INVESTMENT CORPORATION,2015-07-01,2016-06-30,2016,None,+1 (352) 392-1693,WWW.UFICO.UFL.EDU,4510 NW 6TH PLACE 2ND FLOOR,GAINESVILLE,...,None,None,None,None,NaN,0.0,201226494_201606,2015v3.0,201710759349300211,NDk5NTE1Mzk4NzIxODczMjQzMg
3,132548181,A Philip Randolph Institute,2015-09-01,2016-08-31,2016,None,+1 (202) 508-3710,www.apri.org,815 16th Street,Washington,...,None,None,None,None,NaN,0.0,132548181_201608,2015v3.0,201711949349300911,NzE1NjIzMjAyMTQ2NDUwNDczNA
4,222135712,MANALAPAN SOCCER CLUB INC CO HOWARD S KRANT,2016-01-01,2016-12-31,2016,None,+1 (732) 745-8800,None,733 ROUTE 35 NORTH,OCEAN,...,None,None,None,None,NaN,0.0,222135712_201612,2016v3.0,201733199349303373,NTE2ODAyODAwNzMwODg2MDc


In [34]:
by_city = spark.sql('''
SELECT city, state, COUNT(name), SUM(revenue_amt)
FROM master
GROUP BY city, state
''')

In [38]:
by_city.show(5)

+------------+-----+-----------+----------------+
|        city|state|count(name)|sum(revenue_amt)|
+------------+-----+-----------+----------------+
| SUN PRAIRIE|   WI|        190|        25720229|
|ELKHART LAKE|   WI|         25|         1369659|
|  JANESVILLE|   WI|        353|       779232935|
|       BOWIE|   MD|        738|        78094109|
| KING GEORGE|   VA|        100|        91884351|
+------------+-----+-----------+----------------+
only showing top 5 rows



In [39]:
rexburg = spark.sql('''
SELECT *
FROM master
WHERE city == 'REXBURG'
''')

In [40]:
rexburg.count()

77

In [47]:
rexburg.toPandas().sort_values("income_amt", ascending = False)

,ein,name,ico,street,city,state,zip,group,subsection,affiliation,...,asset_cd,income_cd,filing_req_cd,pf_filing_req_cd,acct_pd,asset_amt,income_amt,revenue_amt,ntee_cd,sort_name
64,820423853,ROGER & SYBIL FERGUSON CHARITABLE FOUNDATION,None,PO BOX 519,REXBURG,ID,83440-0519,0,3,3,...,7,6,0,1,12,9731654.0,3504163.0,NaN,None,None
44,820327558,EAST CENTRAL IDAHO PLANNING AND DEVELOPMENT AS...,None,299 E 4TH N,REXBURG,ID,83440-1659,0,4,3,...,8,6,1,0,9,23506180.0,2652724.0,2652724.0,None,None
42,273465046,RESEARCH AND BUSINESS DEVELOPMENT CENTER INC,% TAYLOR WOODS,35 N 1ST E STE 3,REXBURG,ID,83440-1559,0,3,3,...,4,6,1,0,12,275394.0,1042755.0,1042755.0,E22,None
72,820412188,FAMILY CRISIS CENTER,None,16 E MAIN ST,REXBURG,ID,83440-1927,0,3,3,...,4,5,1,0,12,332959.0,796213.0,796213.0,E99Z,None
35,820475833,EAST-CENTRAL IDAHO DEVELOPMENT COMPANY,None,299 E 4TH N,REXBURG,ID,83440-1659,0,4,3,...,6,5,1,0,9,2557772.0,709274.0,709274.0,S43,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,851033510,SNAKE RIVER STORYTELLING,None,240 S 3RD E,REXBURG,ID,83440-2206,0,3,3,...,0,0,2,0,12,NaN,NaN,NaN,A24,None
27,820207699,BRIGHAM YOUNG UNIVERSITY-IDAHO,% LAYNE HYMAS,525 S CENTER ST,REXBURG,ID,83460-0004,0,3,3,...,0,0,13,0,12,NaN,NaN,NaN,None,None
38,261721201,MADISON COUNTY FRATERNAL ORDER OF POLICE LODGE...,None,PO BOX 893,REXBURG,ID,83440-0893,0,8,3,...,0,0,2,0,12,NaN,NaN,NaN,None,None
41,832627768,SUGAR-SALEM CLUB VOLLEYBALL ASSOCIATION,% LISA PANNELL,6631 W 6000 N,REXBURG,ID,83440-3006,0,3,3,...,0,0,2,0,12,NaN,NaN,NaN,N60,SSCVA


In [23]:
spark.sql('SHOW TABLES').show()

+--------+---------------+-----------+
|database|      tableName|isTemporary|
+--------+---------------+-----------+
|        |     contractor|       true|
|        |   exec_charity|       true|
|        |exec_foundation|       true|
|        |     foundation|       true|
|        |     governance|       true|
|        |          grant|       true|
|        |         master|       true|
+--------+---------------+-----------+



In [25]:
spark.sql('''
SELECT *
FROM exec_foundation
WHERE ein == 820423853
''').toPandas()

,ein,period,person_id,title,hours,compensation,benefits,allowances,org_name,subsection,...,state,zip,phone,website_inspect,assets,liabilities,expenses_and_disbursements,revenue_pf,form_type,efile_id
0,820423853,201712,kI-XW50y_4sWewrvnLngxQ,Trustee,8.0,24000,NaN,NaN,ROGER & SYBIL FERGUSON CHARITABLE FOUND CO WAD...,501(c)(3),...,ID,83440,2083563716,N/A,1218255,1,214051,28853,990PF,201842989349100109
1,820423853,201712,WxDw2ckZO81Jc-EspXCshg,Trustee,8.0,24000,NaN,NaN,ROGER & SYBIL FERGUSON CHARITABLE FOUND CO WAD...,501(c)(3),...,ID,83440,2083563716,N/A,1218255,1,214051,28853,990PF,201842989349100109
2,820423853,201712,Ui7rkycMNsnzUOKhwgd-kA,Trustee,8.0,24000,NaN,NaN,ROGER & SYBIL FERGUSON CHARITABLE FOUND CO WAD...,501(c)(3),...,ID,83440,2083563716,N/A,1218255,1,214051,28853,990PF,201842989349100109
